In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, TFBertModel
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [3]:
cwd = os.getcwd()
cwd

'c:\\Users\\LENOVO\\GitHub\\capstone_bangkit'

In [4]:
test_model = tf.saved_model.load('model_3_2023-11-08')

In [5]:
test_model

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject at 0x21ab483ca00>

In [6]:
test = os.path.join(cwd, 'content')
os.listdir(test)

['cnn_content_2023-11-11.csv']

In [7]:
real_df = pd.read_csv(os.path.join(test, 'cnn_content_2023-11-11.csv'))
real_df

,berita,judul,sumber,tanggal,jenis
0,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...,MKMK Perintahkan Saldi Isra Gelar Pemilihan Pe...,CNN Indonesia,"Selasa, 07 Nov 2023 18:27 WIB",nasional
1,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...,"Putusan MKMK: Anwar Usman Langgar Etik Berat, ...",CNN Indonesia,"Selasa, 07 Nov 2023 18:20 WIB",nasional
2,Mahfud lekat dengan inisial MD yang tersemat d...,Asal Usul MD di Buntut Nama Mahfud,CNN Indonesia,"Kamis, 02 Nov 2023 16:17 WIB",nasional
3,Perebutan tiket atau yang populer dengan istil...,Mereka yang Tersingkir di War Tiket Cawapres 2024,CNN Indonesia,"Jumat, 27 Okt 2023 10:12 WIB",nasional
4,Riuh Pemilu 2024 tak cuma seputar capres-cawap...,Pertarungan Caleg di Dapil Neraka Jakarta II,CNN Indonesia,"Rabu, 11 Okt 2023 07:15 WIB",nasional
5,Menteri Kesehatan (Menkes) Budi Gunadi Sadikin...,Menkes Susun UU Baru: Pengguna Napza yang Saki...,CNN Indonesia,"Selasa, 07 Nov 2023 18:03 WIB",nasional
6,Bakal calon presiden Koalisi Perubahan Anies B...,Anies Respons Tim Prabowo-Gibran Gaet Khofifah...,CNN Indonesia,"Selasa, 07 Nov 2023 17:54 WIB",nasional
7,Menteri Pertahanan Prabowo Subianto menyebut a...,Prabowo Sebut Ada Menteri Neoliberal di Kabine...,CNN Indonesia,"Selasa, 07 Nov 2023 17:45 WIB",nasional
8,Sembilan hakim Mahkamah Konstitusi (MK) dikena...,9 Hakim MK Kena Sanksi Teguran Lisan Buntut RP...,CNN Indonesia,"Selasa, 07 Nov 2023 17:40 WIB",nasional
9,Undang-undang Nomor 7 Tahun 2017 tentang Pemil...,Pemilu: Pemda Dilarang Bertindak yang Merugika...,CNN Indonesia,"Senin, 06 Nov 2023 10:42 WIB",nasional


In [8]:
df = real_df[['berita']]

In [26]:
test_twt = {
    'berita' : ["lewat di tl aku, dan aku kaget. Aku kira RS ini dibangunnya sama pemerintah kita ternyata pembangunannya dari uang donasi masyarakat kita toh aku nangis banget pas liat videonya juga", "Jog dalam kota kecepatan 80km/jam dan nerobos lampu merah. Dimf bawah ini juga ada cctv kalau ojol berhenti dipinggir anda tabrak, giliran diminta ganti rugi merasa diperas. Playing victim sekali ya wkwkwk", "Peta hubungan diplomatik Israel dengan negara-negara dunia. Hanya ada satu negara dengan perekonomian raksasa ber-GDP di atas $1 triliun yang belum pernah menjalin relasi sama Israel. Yep, Indonesia."] 
}

df_test_twt = pd.DataFrame(test_twt)
df_test_twt

df = pd.concat([real_df[['berita']], df_test_twt], ignore_index=True)
df

,berita
0,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...
1,Majelis Kehormatan Mahkamah Konstitusi (MKMK) ...
2,Mahfud lekat dengan inisial MD yang tersemat d...
3,Perebutan tiket atau yang populer dengan istil...
4,Riuh Pemilu 2024 tak cuma seputar capres-cawap...
5,Menteri Kesehatan (Menkes) Budi Gunadi Sadikin...
6,Bakal calon presiden Koalisi Perubahan Anies B...
7,Menteri Pertahanan Prabowo Subianto menyebut a...
8,Sembilan hakim Mahkamah Konstitusi (MK) dikena...
9,Undang-undang Nomor 7 Tahun 2017 tentang Pemil...


In [27]:
# datapath = os.path.join(cwd, 'dataset')

# df = pd.read_excel(os.path.join(datapath, 'all_cleaned.xlsx'))
# df = df[['berita', 'label']]

df.drop_duplicates(inplace=True)
df.dropna(inplace = True)

df_test = df

max_len = 70
model_name = 'indobenchmark/indobert-base-p1'
tokenizer = AutoTokenizer.from_pretrained(model_name)

X_train = tokenizer(
    text=df_test['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_train

{'input_ids': <tf.Tensor: shape=(23, 70), dtype=int32, numpy=
array([[    2,  6810,  9293, ...,   680, 30471,     3],
       [    2,  6810,  9293, ...,  4887,  7343,     3],
       [    2, 20469, 26243, ...,   126,  1328,     3],
       ...,
       [    2,  2145,    26, ...,     0,     0,     0],
       [    2,  3681,   112, ...,     0,     0,     0],
       [    2,  2577,  1314, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(23, 70), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}

In [33]:
X_train[20]

Encoding(num_tokens=70, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [45]:
df.iloc[20]['berita'].split()

['lewat',
 'di',
 'tl',
 'aku,',
 'dan',
 'aku',
 'kaget.',
 'Aku',
 'kira',
 'RS',
 'ini',
 'dibangunnya',
 'sama',
 'pemerintah',
 'kita',
 'ternyata',
 'pembangunannya',
 'dari',
 'uang',
 'donasi',
 'masyarakat',
 'kita',
 'toh',
 'aku',
 'nangis',
 'banget',
 'pas',
 'liat',
 'videonya',
 'juga']

In [43]:
X_train['input_ids'][20]

<tf.Tensor: shape=(70,), dtype=int32, numpy=
array([    2,  2145,    26,  9480,   304, 30468,    41,   304,  7950,
       30470,   304,  5157,  3935,    92,  3887,    57,   500,   877,
         219,  1369, 24734,    98,   988, 21781,   552,   219,  8027,
         304, 15089,  2174,   280,  4961, 21370,   186,     3,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0])>

In [38]:
X_train['attention_mask'][20]

<tf.Tensor: shape=(70,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0])>

In [28]:
input_ids = tf.TensorSpec.from_tensor(X_train['input_ids'], name = 'input_ids')
attention_mask = tf.TensorSpec.from_tensor(X_train['attention_mask'], name = 'attention_mask')

In [29]:
predicted = test_model([X_train['input_ids'], X_train['attention_mask']])
y_predicted = [1 if pred >= 0.5 else 0 for pred in predicted]
y_predicted, predicted

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 <tf.Tensor: shape=(23, 1), dtype=float32, numpy=
 array([[0.01038714],
        [0.00535888],
        [0.11358348],
        [0.04632161],
        [0.00635119],
        [0.02014987],
        [0.00183882],
        [0.05581692],
        [0.05219695],
        [0.21371216],
        [0.02636046],
        [0.1612767 ],
        [0.01700469],
        [0.26391745],
        [0.02105645],
        [0.00555589],
        [0.01810883],
        [0.03165722],
        [0.00152159],
        [0.01148606],
        [0.9994536 ],
        [0.99762005],
        [0.41494957]], dtype=float32)>)

In [ ]:
df_test['label']

KeyError: 'label'